In [ ]:
# Text classification model with tensorflow
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfsd

In [ ]:
### Split the dataset into train and test
train_data, test_data, validation_data=tfsd.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteVX24AI/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteVX24AI/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteVX24AI/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
### Print first 10 labels from dataset.

train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [ ]:
### Data Exploration
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [ ]:
### Tensorflow-hub is a pretrained model.
### Build text classification model.
### TensorFlow Hub is a library for reusable machine learning modules,
### where a module contains a self-contained piece of a TensorFlow Graph along with its weights and assets.

embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [ ]:
### Build the model on the complete dataset
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 20)                400020    
                                                                 
 dense_2 (Dense)             (None, 16)                336       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400373 (1.53 MB)
Trainable params: 400373 (1.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
### Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
#Train the text classification model,
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/20
30/30 [==============================] - 4s 130ms/step - loss: 0.0105 - accuracy: 0.9995 - val_loss: 0.6355 - val_accuracy: 0.8414
Epoch 2/20
30/30 [==============================] - 4s 130ms/step - loss: 0.0096 - accuracy: 0.9995 - val_loss: 0.6469 - val_accuracy: 0.8410
Epoch 3/20
30/30 [==============================] - 5s 160ms/step - loss: 0.0089 - accuracy: 0.9997 - val_loss: 0.6578 - val_accuracy: 0.8402
Epoch 4/20
30/30 [==============================] - 4s 130ms/step - loss: 0.0083 - accuracy: 0.9997 - val_loss: 0.6723 - val_accuracy: 0.8398
Epoch 5/20
30/30 [==============================] - 4s 130ms/step - loss: 0.0076 - accuracy: 0.9997 - val_loss: 0.6812 - val_accuracy: 0.8393
Epoch 6/20
30/30 [==============================] - 4s 131ms/step - loss: 0.0070 - accuracy: 0.9999 - val_loss: 0.6926 - val_accuracy: 0.8393
Epoch 7/20
30/30 [==============================] - 3s 103ms/step - loss: 0.0065 - accuracy: 0.9999 - val_loss: 0.7029 - val_accuracy: 0.8387
Epoch 

In [ ]:
#### Evaluate the model
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

20/20 - 1s - loss: 0.7264 - accuracy: 0.8581 - 713ms/epoch - 36ms/step
loss: 0.726
accuracy: 0.858


In [ ]:
# Our text  classification model achieved an accuracy of 85%, which is generally accepted.
model.predict(["This is the best movie I have ever seen"])

1/1 [==============================] - 0s 160ms/step


array([[-1.0521957]], dtype=float32)

In [ ]:
model.predict('Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fel')

SyntaxError: unterminated string literal (detected at line 1) (<ipython-input-40-0cac04726ec8>, line 1)